In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
label_df = pd.read_csv("/home/kongge/projects/Phd/data/all_labels.csv")
prs_df = pd.read_csv('/home/kongge/projects/Phd/data/Only_PGS.csv')
data = pd.merge(prs_df, label_df, left_on='FID', right_on='eid')

In [3]:
prs_df

,FID,AF,CAD,DCM,HT,ISS,VTE,STK,HF,IH,...,GM,HB,HIR,IRM,ISM,NMR,PM,TSH,TM,AST
0,1000011,-0.141637,-0.477495,-0.195173,0.931986,-0.894940,-0.473692,-0.317333,-1.737386,0.020924,...,0.079802,0.695521,1.277698,1.504517,-1.692002,-1.071869,-1.125821,-0.288634,1.733040,-0.264657
1,1000026,-1.473287,0.367636,-1.566975,0.309153,-0.882619,1.044783,-0.128957,-1.362442,-0.811806,...,0.026687,1.076239,0.824551,0.690095,0.018237,-1.071869,-0.092784,0.843544,0.983909,-0.596751
2,1000044,0.585503,2.575461,1.860125,1.589465,0.237156,0.309952,0.659745,-0.803434,-1.499108,...,-1.726094,1.125755,-0.352180,-0.544924,-0.093467,-0.784642,-0.278345,0.444989,0.895999,0.153730
3,1000058,0.847278,1.352384,-1.004531,1.490209,1.398947,-1.508763,1.176881,-1.164744,1.711949,...,0.132917,0.406527,-0.551243,-0.256447,1.001336,-1.071869,1.090921,-0.241189,0.366977,-0.084286
4,1000116,-0.805020,-1.414212,-1.471695,-0.624252,-0.147236,-1.015938,-0.131356,-1.573774,-0.979470,...,1.752911,-0.531077,0.120916,-0.138439,0.928665,0.565118,-0.054315,-1.173920,-0.205888,-0.852839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336974,6023498,-0.779058,0.723359,0.627861,0.608226,0.502332,-0.416751,0.846122,1.364426,0.426492,...,0.026687,-0.632701,0.033418,0.165458,-0.954425,0.038104,-1.563399,0.586071,0.152152,-0.977810
336975,6023514,1.169570,-1.041206,0.685736,0.225246,0.006642,1.845830,-1.305210,-0.128534,0.239780,...,1.062422,-0.736056,0.243756,1.266113,0.187791,-1.071869,-0.570433,-0.486148,0.664426,-0.221420
336976,6023529,1.536596,-0.944525,-1.870310,-1.580533,0.422397,0.295691,1.207604,-0.898874,0.581479,...,-0.345115,0.528340,1.535187,0.872027,-0.182953,1.675091,0.401696,0.794166,-2.078717,0.779929
336977,6023541,-0.441969,-1.006966,2.482164,1.721099,0.079389,-1.418951,0.887717,0.580452,0.091717,...,-0.159214,-1.723317,-1.550319,-0.558635,0.989086,-1.071869,0.002586,-0.034688,-0.387663,0.904508


In [4]:
label_df

,eid,Class_A00,Class_A01,Class_A02,Class_A03,Class_A04,Class_A05,Class_A06,Class_A07,Class_A08,...,Class_I48,Class_I63,Class_I10,Class_F00,Class_F20,Class_C50,Class_C18,Class_C56,Class_J45,Class_M05
0,1000011,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000026,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1000044,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000058,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,1000116,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336974,6023498,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336975,6023514,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
336976,6023529,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
336977,6023541,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [5]:
positive_counts = label_df.iloc[:, 1:].sum()
positive_counts_df = positive_counts.to_frame(name='Positive_Counts')
positive_counts_df

,Positive_Counts
Class_A00,18
Class_A01,84
Class_A02,554
Class_A03,147
Class_A04,5845
...,...
Class_C50,12694
Class_C18,7027
Class_C56,1359
Class_J45,49477


In [6]:
positive_counts_df_order = positive_counts_df.sort_values(by='Positive_Counts', ascending=False)

In [7]:
positive_counts_df_order

,Positive_Counts
Class_I10,135881
Class_J45,49477
Class_I25,39523
Class_I48,29871
Class_F17,29865
...,...
Class_D60,12
Class_D55,11
Class_E12,6
Class_G07,3


In [8]:
label_selected = label_df['Class_I10']

In [9]:
label_selected.head()
# TODO 筛选出列 11,8, 5:00

0    0
1    1
2    0
3    1
4    0
Name: Class_I10, dtype: int64

In [10]:
label_selected.shape #作为新的y

(336979,)

In [11]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
features = prs_df.drop(columns=['FID'])
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)
prs_df_normalized = pd.DataFrame(features_normalized, columns=features.columns)

In [12]:
prs_df_normalized['FID'] = prs_df['FID']

In [13]:
prs_df_normalized

,AF,CAD,DCM,HT,ISS,VTE,STK,HF,IH,AAA,...,HB,HIR,IRM,ISM,NMR,PM,TSH,TM,AST,FID
0,0.432506,0.461849,0.456030,0.640938,0.421417,0.458939,0.444609,0.268111,0.530358,0.686878,...,0.592280,0.654823,0.663314,0.256496,0.000000,0.426032,0.501385,0.710446,0.467130,1000011
1,0.283769,0.553655,0.315785,0.573452,0.422716,0.608267,0.463927,0.309186,0.437324,0.662286,...,0.633660,0.611275,0.573128,0.463352,0.000000,0.534133,0.629220,0.641481,0.430507,1000026
2,0.513724,0.793488,0.666151,0.712178,0.540801,0.536003,0.544809,0.370426,0.360538,0.620975,...,0.639042,0.498190,0.436367,0.449841,0.087730,0.514715,0.584219,0.633388,0.513268,1000044
3,0.542962,0.660626,0.373286,0.701423,0.663316,0.357149,0.597843,0.330844,0.719281,0.612873,...,0.560868,0.479060,0.468312,0.582258,0.000000,0.658001,0.506742,0.584686,0.487020,1000058
4,0.358410,0.360095,0.325526,0.472314,0.500265,0.405614,0.463681,0.286034,0.418592,0.540207,...,0.458958,0.543655,0.481380,0.573469,0.500000,0.538158,0.401427,0.531947,0.402267,1000116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336974,0.361310,0.592296,0.540172,0.605857,0.568765,0.464538,0.563923,0.607916,0.575668,0.529904,...,0.447913,0.535246,0.515032,0.345707,0.339029,0.380242,0.600149,0.564909,0.388485,6023498
336975,0.578960,0.400614,0.546089,0.564360,0.516492,0.687043,0.343300,0.444361,0.554808,0.404555,...,0.436679,0.555460,0.636914,0.483859,0.000000,0.484150,0.479084,0.612069,0.471898,6023514
336976,0.619955,0.411116,0.284774,0.368698,0.560335,0.534601,0.600993,0.359970,0.592983,0.499185,...,0.574108,0.679567,0.593275,0.439017,0.839029,0.585877,0.623645,0.359533,0.582323,6023529
336977,0.398961,0.404333,0.729745,0.726441,0.524163,0.365981,0.568188,0.522031,0.538267,0.529901,...,0.329372,0.383048,0.434849,0.580777,0.000000,0.544113,0.530059,0.515213,0.596061,6023541


In [14]:
X_tensor = torch.tensor(prs_df_normalized.iloc[:,:-1].values, dtype=torch.float32)

In [15]:
X_tensor.size()

torch.Size([336979, 80])

In [26]:
y_tensor = torch.tensor(label_selected.values, dtype=torch.float32)

In [27]:
y_tensor = y_tensor.unsqueeze(1)

In [28]:
y_tensor.size()

torch.Size([336979, 1])

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

In [30]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_size)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [31]:
input_size = X_train.shape[1]
output_size = y_train.shape[1]
model = MLP(input_size, output_size)

In [32]:
input_size

80

In [33]:
# pos_weight = torch.tensor([len(y_train) / y_train.sum().item()], dtype=torch.float32)
# criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [34]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=5096, shuffle=True)
# from torch.utils.data import WeightedRandomSampler
# pos_weight = torch.tensor([len(y_train) / y_train.sum().item()], dtype=torch.float32)
#
# class_weights = [1.0 if label == 0 else pos_weight.item() for label in y_train.numpy()]
# sampler = WeightedRandomSampler(class_weights, len(class_weights), replacement=True)
# train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler)

In [36]:
num_epochs = 30
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/30], Loss: 0.6252
Epoch [2/30], Loss: 0.6251
Epoch [3/30], Loss: 0.6251
Epoch [4/30], Loss: 0.6249
Epoch [5/30], Loss: 0.6246
Epoch [6/30], Loss: 0.6247
Epoch [7/30], Loss: 0.6246
Epoch [8/30], Loss: 0.6246
Epoch [9/30], Loss: 0.6245
Epoch [10/30], Loss: 0.6244
Epoch [11/30], Loss: 0.6243
Epoch [12/30], Loss: 0.6241
Epoch [13/30], Loss: 0.6240
Epoch [14/30], Loss: 0.6238
Epoch [15/30], Loss: 0.6238
Epoch [16/30], Loss: 0.6237
Epoch [17/30], Loss: 0.6240
Epoch [18/30], Loss: 0.6237
Epoch [19/30], Loss: 0.6238
Epoch [20/30], Loss: 0.6234
Epoch [21/30], Loss: 0.6233
Epoch [22/30], Loss: 0.6234
Epoch [23/30], Loss: 0.6233
Epoch [24/30], Loss: 0.6231
Epoch [25/30], Loss: 0.6231
Epoch [26/30], Loss: 0.6231
Epoch [27/30], Loss: 0.6232
Epoch [28/30], Loss: 0.6229
Epoch [29/30], Loss: 0.6234
Epoch [30/30], Loss: 0.6230


In [37]:
model.eval()
from sklearn import metrics
with torch.no_grad():
    y_pred = model(X_test).numpy()
    y_true = y_test.numpy()
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
    aupr = metrics.average_precision_score(y_true, y_pred)
    roc_auc = metrics.auc(fpr, tpr)

In [38]:
roc_auc, aupr # ROC AUC 值较高（0.919），而 AUPR 值却非常低（0.00018）。模型虽然整体分类能力较强（ROC AUC 高），但在正类（即标签为 1 的样本）上的表现较差

(0.6849201154493059, 0.5885584529795603)

In [35]:
# model.eval()
# with torch.no_grad():
#     y_pred = model(X_test)  # 预测概率
#     # y_pred_binary = (y_pred >= 0.5).float()

In [36]:
# selected_columns

['Class_I10',
 'Class_A09',
 'Class_F05',
 'Class_D51',
 'Class_F06',
 'Class_G12',
 'Class_G06',
 'Class_A03',
 'Class_I06',
 'Class_D57']

In [37]:
# from sklearn.metrics import roc_auc_score, average_precision_score
#
# y_true = y_test.numpy()
# y_scores = y_pred.numpy()
#
# auc_scores = {}
# aupr_scores = {}
#
# for i in range(y_true.shape[1]):
#     label_name = selected_columns[i]
#     auc = roc_auc_score(y_true[:, i], y_scores[:, i])
#     auc_scores[label_name] = auc
#
#     aupr = average_precision_score(y_true[:, i], y_scores[:, i])
#     aupr_scores[label_name] = aupr
#     print(f"{label_name} 的 AUC: {auc:.4f}, AUPR: {aupr:.4f}")
#
# metrics_df = pd.DataFrame({
#     'AUC': auc_scores,
#     'AUPR': aupr_scores
# })


Class_I10 的 AUC: 0.6792, AUPR: 0.5833
Class_A09 的 AUC: 0.5041, AUPR: 0.0538
Class_F05 的 AUC: 0.4841, AUPR: 0.0172
Class_D51 的 AUC: 0.5119, AUPR: 0.0080
Class_F06 的 AUC: 0.4918, AUPR: 0.0034
Class_G12 的 AUC: 0.5390, AUPR: 0.0020
Class_G06 的 AUC: 0.5192, AUPR: 0.0009
Class_A03 的 AUC: 0.5135, AUPR: 0.0005
Class_I06 的 AUC: 0.3989, AUPR: 0.0002
Class_D57 的 AUC: 0.5014, AUPR: 0.0001
